# NLP: Trabajo Práctico N°1
**Integrantes**
- López Ceratto, Julieta : L-3311/1
- Crenna, Giuliano : 

# Importamos librerías necesarias

In [1]:
from rqst import *

C:\Users\julil\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


# Carga de datasets

In [2]:
dataset_juegos = pd.read_csv('./src/bgg_database.csv')
dataset_peliculas = pd.read_csv('./src/IMDB-Movie-Data.csv')

## Dataset libros

In [94]:
ruta_url = 'https://www.gutenberg.org/browse/scores/top1000.php#books-last1'
response = requests.get(ruta_url, verify=False)
soup = BeautifulSoup(response.text, 'html.parser')
ol = soup.findAll('ol')
libros = []
for libro in ol:
    enlaces = libro.findAll('li')
    for enlace in enlaces:
        libros.append(enlace.text)

C:\Users\julil\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.gutenberg.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [96]:
libros = [libro for libro in libros if libro != '']

In [97]:
# Expresión regular para extraer las partes
patron_titulo_principal = r'^(.*?)(?:;?\s?Or,|\sby|\s\()'  # Captura todo hasta "Or," o "by" o "(" si no hay más
patron_titulo_secundario = r';?\s?Or,?\s(.*?)\sby'  # Captura el título secundario si está
patron_autor = r'by\s(.*?)\s\('  # Captura el autor si está
patron_n_ref = r'\((\d+)\)'  # Captura el número entre paréntesis
patron_link = r'\)\s(.*)'  # Captura todo después del paréntesis final para el enlace
titulo_principal = []
titulo_secundario = []
autor = []
n_ref = []
for libro in libros:
    # Extraer título principal
    titulo = re.search(patron_titulo_principal, libro)
    titulo_principal.append(titulo.group(1) if titulo else "")

    # Extraer título secundario (si existe)
    secundario = re.search(patron_titulo_secundario, libro)
    titulo_secundario.append(secundario.group(1) if secundario else "")

    # Extraer autor (si existe)
    autor_libro = re.search(patron_autor, libro)
    autor.append(autor_libro.group(1) if autor_libro else "")

    # Extraer número de referencia
    ref = re.search(patron_n_ref, libro)
    n_ref.append(ref.group(1) if ref else "")
dataset_libros = pd.DataFrame({
    'Titulo Principal': titulo_principal,
    'Titulo Secundario': titulo_secundario,
    'Autor': autor,
    'N° Ref': n_ref,
})

In [98]:
dataset_libros

,Titulo Principal,Titulo Secundario,Autor,N° Ref
0,Frankenstein,The Modern Prometheus,Mary Wollstonecraft Shelley,6676
1,Pride and Prejudice,,Jane Austen,2961
2,Moby Dick,The Whale,"Dick; Or, The Whale by Herman Melville",2753
3,Romeo and Juliet,,William Shakespeare,2648
4,The Scarlet Letter,,Nathaniel Hawthorne,2429
...,...,...,...,...
5995,"Laclos, Choderlos de",,,2704
5996,"Brandes, Georg",,,2698
5997,"Del Rey, Lester",,,2697
5998,"Hauff, Wilhelm",,,2687


## Traducciones de Datasets